[<img src="../assets/images/Logo.png" width="64" align="center">](https://github.com/a-fuchs/jupyter-micropython-extended-kernel) &emsp;  [README](../README.md) / [NoteBooks](./Index.ipynb) | **Web REPL**

---

# Web REPL


If all is set up, a connection to the controller could be established:

```
%websocketconnect --password abcd1234 --verbose ws://192.168.4.1:8266
```


---
## Setup password and copy moduls to the controller

In [ ]:
%serialconnect

---
### Setup password in `webrepl_cfg.py`

In [ ]:
#import webrepl_setup

@fuh 2025-03-16: does not work (no field to put in answer appears),  
so set it manualy:

In [ ]:
%sendtofile webrepl_cfg.py
PASS = 'abcd1234'

Check if it is set correctly:

```sh
%fetchfile --binary --load webrepl_cfg.py
```
should result in
***
```python
#%fetchfile --binary --load webrepl_cfg.py

PASS = 'abcd1234'
```
---
```sh
Fetched 17=17 bytes from webrepl_cfg.py.
```

In [ ]:
%fetchfile --binary --load webrepl_cfg.py

Copy this code to main.py and restart the ESP32, or execute direct:

---
### class WebServer, class REPLManager

This class is copied to the controller so we can use it with `import replmanager`.

In [ ]:
%sendtofile "/replmanager.py"
#%%writefile replmanager.py
import network
import sys
import machine
import time
import binascii

class WebServer :
    def __init__( self, bLog = True ):
        self.ssid       = None
        self.ipAddress  = None
        self.subnetMask = None
        self.gateway    = None
        self.dnsServer  = None
        self.macAddress = None
        self.bLog       = bLog
        
    # attension: _password has to be at least 8 chars long,
    # otherwise error 'OSError: can't set AP config' is thrown
    def setupAsAccessPoint( self, _ssid, _password = None, authmode = network.AUTH_WPA2_WPA3_PSK ): # AUTH_OPEN, AUTH_WPA2_WPA3_PSK
        self.log( "TRY: Setup as AccessPoint ..." )

        if not _password:
            try:
                import webrepl_cfg
                _password = webrepl_cfg.PASS
            except:
                _password = 'abcd1234'

        self.ssid = _ssid
        
        accessPoint = network.WLAN( network.AP_IF )
        accessPoint.active( True )
        accessPoint.config( essid = _ssid, password = _password, authmode = authmode )

        for iIndex in range( 100 ) :
            if accessPoint.active() == False :
                time.sleep_ms(100)
            else :
                break

        if accessPoint.active() == True :
            print( accessPoint.ifconfig() )
            self.setInfoFrom( accessPoint )
            self.log( "DONE: Setup as AccessPoint" )
            return True
        
        else :
            self.log( "ERROR: Faild setting up as AccessPoint in 10 s" )
            return False

    def setInfoFrom( self, accessPoint ):
        self.ipAddress, self.subnetMask, self.gateway, self.dnsServer = accessPoint.ifconfig() # [IP address, subnet mask, gateway, DNS server]
        self.macAddress = binascii.hexlify( network.WLAN().config('mac'),':' ).decode()

    def log( self, *args, **kwargs ):
        if self.bLog == True:
            print( *args, **kwargs )

import webrepl

class REPLManager:
    webServer = None
    
    @staticmethod
    def setWebREPL( ssid = "ESP32-WiFi-000" ):
        try:
            REPLManager.webServer = WebServer()
            REPLManager.webServer.setupAsAccessPoint( ssid )
            webrepl.start()
        except BaseException as e:
            print( e )
            
    @staticmethod
    def setDefaultREPL():
        try:
            webrepl.stop()
        except BaseException as e:
            print( e )
                
        os.dupterm( None )

---
### main.py

The `main.py` is called every time the controller starts.  

It starts the `Web-REPL` if the RST-Button (more precise the button defined with `_BTN_STOP`) is not pressed within 2 seconds (`_WAIT_TIME_MS=2000`), otherwise it does nothing, so you can connect over USB.

In [35]:
%sendtofile "/main.py"
_BTN_STOP     = 0
_WAIT_TIME_MS = 2000

import time
import machine
import replmanager

btnStop = machine.Pin( _BTN_STOP, machine.Pin.IN, machine.Pin.PULL_UP )

bStartWebREPL = True
lastTimeMs    = time.ticks_ms()

while time.ticks_diff( time.ticks_ms(), lastTimeMs ) < _WAIT_TIME_MS:
    if btnStop.value() == 0:
        bStartWebREPL = False
        break
    time.sleep_ms(50)
    
if bStartWebREPL:
    try:
        replmanager.REPLManager.setWebREPL()
    except:
        pass

import gc
gc.collect()


Sent 26 lines (505 bytes) to /main.py.


Now connect to the WLAN with ssid `ESP32-WiFi-000` and than connect via the Micropython USB - kernel:

---
## Connect to controller and test
If all is set up, a connection to the controller could be established after connecting to the WiFi of the ESP-Accesspoint with the ssid `ESP32-WiFi-000`.

```
%websocketconnect --password abcd1234 --verbose ws://192.168.4.1:8266
```

In [ ]:
%websocketconnect --password abcd1234 --verbose ws://192.168.4.1:8266

In [ ]:
print( "Hello world over WiFi!" )